In [1]:
import importlib
import dill as pickle
from tqdm import tqdm
from tqdm import trange

import pentago
from pentago import agent, memory, minimax, learning_agent, neural_model, monte_carlo, evaluate, game, multi

import ray
ray.init()

2019-04-12 08:34:51,210	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-12_08-34-51_9356/logs.


2019-04-12 08:34:51,339	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:28578 to respond...


2019-04-12 08:34:51,456	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:23816 to respond...


2019-04-12 08:34:51,460	INFO services.py:760 -- Starting Redis shard with 4.2 GB max memory.


2019-04-12 08:34:51,482	INFO services.py:1384 -- Starting the Plasma object store with 6.3 GB memory using /dev/shm.


{'node_ip_address': None,
 'redis_address': '10.60.55.201:28578',
 'object_store_address': '/tmp/ray/session_2019-04-12_08-34-51_9356/sockets/plasma_store',
 'webui_url': None,
 'raylet_socket_name': '/tmp/ray/session_2019-04-12_08-34-51_9356/sockets/raylet'}

In [2]:
view = memory.MemoryView()
round = 0

In [3]:
multi.self_play_games(minimax.MinimaxSearchAgent(depth=2),
                      view,
                      1000,
                      verbose=True,
                      num_workers=3)

528.8494734764099 seconds taken, 0.5288494734764099 per game


In [34]:
pickle.dump(view, open('initial_view.pkl', mode='wb'))

In [4]:
view = pickle.load(open('initial_view.pkl', mode='rb'))

In [4]:
model_params = {'dense_regularization_const': 1e-2,
                'regularization_const': 1e-4,
                'num_layers': 5,
                'kernel_size': 5,
                'num_filters': 64}
current_agent = learning_agent.NeuralAgent(model_params=model_params)
best_agent = learning_agent.NeuralAgent(model_params=model_params)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
while True:
    round += 1
    multi.self_play_games(current_agent,
                          view,
                          30,
                          verbose=True,
                          num_workers=3)
    for e in range(4):
        current_agent.fit(view.get_experiences(sample_k=1000), epochs=1)
    if round % 5 == 0:
        wins = multi.tally_wins([minimax.MinimaxSearchAgent(), current_agent], 10, num_workers=3)
        print('Round {}, against minimax: {}'.format(round, wins))
        wins = multi.tally_wins([best_agent, current_agent], 10, num_workers=3)
        print('Round {}, against current best neural: {}'.format(round, wins))
        if wins[-1] > 7:
            print('Ratcheting best agent')
            best_agent.model.set_weights(current_agent.model.get_weights())

547.2369258403778 seconds taken, 18.241230861345926 per game
Train on 3600 samples, validate on 400 samples


 - 5s - loss: 5.3600 - policy_head_loss: 4.3967 - value_head_loss: 0.8308 - val_loss: 5.6990 - val_policy_head_loss: 4.6544 - val_value_head_loss: 0.9127


Train on 3600 samples, validate on 400 samples


 - 5s - loss: 5.3477 - policy_head_loss: 4.3891 - value_head_loss: 0.8270 - val_loss: 5.6458 - val_policy_head_loss: 4.6142 - val_value_head_loss: 0.9005


Train on 3600 samples, validate on 400 samples


...

 - 5s - loss: 5.2301 - policy_head_loss: 4.2429 - value_head_loss: 0.8895 - val_loss: 5.2283 - val_policy_head_loss: 4.2729 - val_value_head_loss: 0.8579


Round 35, against minimax: Counter({1: 9, -1: 1, 0: 0})


Round 35, against current best neural: Counter({-1: 8, 1: 2, 0: 0})
Ratcheting best agent


Exception in thread ray_listen_error_messages:
Traceback (most recent call last):
  File "/home/derick/anaconda3/envs/gai-project/lib/python3.7/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/derick/anaconda3/envs/gai-project/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/derick/anaconda3/envs/gai-project/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/derick/anaconda3/envs/gai-project/lib/python3.7/site-packages/ray/worker.py", line 1758, in listen_error_messages_raylet
    msg = worker.error_message_pubsub_client.get_message()
  File "/home/derick/anaconda3/envs/gai-project/lib/python3.7/site-packages/redis/client.py", line 3135, in get_message
    response = 

KeyboardInterrupt: 

In [14]:
# some kind of epic memory leak going on...
view.temp_memory.clear()
ray.shutdown()
importlib.reload(multi)
ray.init()

2019-04-12 16:25:20,142	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-12_16-25-20_9356/logs.


2019-04-12 16:25:20,267	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:27105 to respond...


2019-04-12 16:25:20,403	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:57312 to respond...


2019-04-12 16:25:20,408	INFO services.py:760 -- Starting Redis shard with 4.2 GB max memory.


2019-04-12 16:25:20,463	INFO services.py:1384 -- Starting the Plasma object store with 6.3 GB memory using /dev/shm.


{'node_ip_address': None,
 'redis_address': '10.39.10.6:27105',
 'object_store_address': '/tmp/ray/session_2019-04-12_16-25-20_9356/sockets/plasma_store',
 'webui_url': None,
 'raylet_socket_name': '/tmp/ray/session_2019-04-12_16-25-20_9356/sockets/raylet'}

In [ ]:
while True:
    round += 1
    multi.self_play_games(current_agent,
                          view,
                          30,
                          verbose=True,
                          num_workers=3)
    for e in range(4):
        current_agent.fit(view.get_experiences(sample_k=1000), epochs=1)
    if round % 5 == 0:
        wins = multi.tally_wins([minimax.MinimaxSearchAgent(), current_agent], 10, num_workers=3)
        print('Round {}, against minimax: {}'.format(round, wins))
        wins = multi.tally_wins([best_agent, current_agent], 10, num_workers=3)
        print('Round {}, against current best neural: {}'.format(round, wins))
        if wins[-1] > 7:
            print('Ratcheting best agent')
            best_agent.model.set_weights(current_agent.model.get_weights())

(pid=1007) WARNING:tensorflow:From /home/derick/anaconda3/envs/gai-project/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=1007) Instructions for updating:
(pid=1007) Colocations handled automatically by placer.
(pid=1001) WARNING:tensorflow:From /home/derick/anaconda3/envs/gai-project/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=1001) Instructions for updating:
(pid=1001) Colocations handled automatically by placer.
(pid=1008) WARNING:tensorflow:From /home/derick/anaconda3/envs/gai-project/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=1008) Instructions

(pid=1001) 2019-04-12 16:25:24.641519: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3502470000 Hz
(pid=1001) 2019-04-12 16:25:24.641684: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x563f2db74b30 executing computations on platform Host. Devices:
(pid=1001) 2019-04-12 16:25:24.641704: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=1007) 2019-04-12 16:25:24.699061: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3502470000 Hz
(pid=1007) 2019-04-12 16:25:24.699211: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55c453a772d0 executing computations on platform Host. Devices:
(pid=1007) 2019-04-12 16:25:24.699224: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=1008) 2019-04-12 16:25:24.748682: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3502470000 Hz
(pid

346.6110875606537 seconds taken, 11.553702918688456 per game
Train on 3600 samples, validate on 400 samples


 - 5s - loss: 5.1070 - policy_head_loss: 4.1230 - value_head_loss: 0.9020 - val_loss: 5.1467 - val_policy_head_loss: 4.1336 - val_value_head_loss: 0.9313


Train on 3600 samples, validate on 400 samples


...

 - 5s - loss: 4.7466 - policy_head_loss: 3.8718 - value_head_loss: 0.8082 - val_loss: 4.8091 - val_policy_head_loss: 3.9368 - val_value_head_loss: 0.8057


Train on 3600 samples, validate on 400 samples


 - 5s - loss: 4.7393 - policy_head_loss: 3.8951 - value_head_loss: 0.7776 - val_loss: 4.7436 - val_policy_head_loss: 3.8752 - val_value_head_loss: 0.8018


Train on 3600 samples, validate on 400 samples


 - 5s - loss: 4.7059 - policy_head_loss: 3.8732 - value_head_loss: 0.7662 - val_loss: 4.7822 - val_policy_head_loss: 3.9100 - val_value_head_loss: 0.8056
